In [1]:
class Polynomial:
    def __init__(self, coefficients, p=2):
        """
        Инициализация полинома с коэффициентами в поле GF(p).
        coefficients: список коэффициентов, начиная со старшего разряда.
        p: простое число, определяющее поле GF(p).
        """
        self.coefficients = [c % p for c in coefficients]
        self.p = p

    def __add__(self, other):
        """Сложение полиномов в поле GF(p)."""
        max_len = max(len(self.coefficients), len(other.coefficients))
        padded_self = [0] * (max_len - len(self.coefficients)) + self.coefficients
        padded_other = [0] * (max_len - len(other.coefficients)) + other.coefficients
        result = [(a + b) % self.p for a, b in zip(padded_self, padded_other)]
        return Polynomial(result, self.p)

    def __mul__(self, other):
        """Умножение полиномов в поле GF(p)."""
        result = [0] * (len(self.coefficients) + len(other.coefficients) - 1)
        for i, a in enumerate(self.coefficients):
            for j, b in enumerate(other.coefficients):
                result[i + j] = (result[i + j] + a * b) % self.p
        return Polynomial(result, self.p)

    def __mod__(self, other):
        """Деление полиномов в поле GF(p) (возвращает остаток)."""
        dividend = self.coefficients[:]
        divisor = other.coefficients[:]
        while len(dividend) >= len(divisor):
            shift = len(dividend) - len(divisor)
            quotient_term = (dividend[0] * pow(divisor[0], -1, self.p)) % self.p
            for i in range(len(divisor)):
                dividend[i + shift] = (dividend[i + shift] - quotient_term * divisor[i]) % self.p
            # Удалить ведущие нули
            while dividend and dividend[0] == 0:
                dividend.pop(0)
        return Polynomial(dividend, self.p)

    def __str__(self):
        """Строковое представление полинома."""
        terms = []
        for i, coeff in enumerate(self.coefficients):
            if coeff != 0:
                power = len(self.coefficients) - i - 1
                term = f"{coeff}" if power == 0 else f"{coeff}x^{power}"
                terms.append(term)
        return " + ".join(terms) if terms else "0"


class ShiftRegister:
    def __init__(self, n, initial_state, feedback_polynomial, p=2):
        """
        Инициализация регистра сдвига.
        n: количество регистров.
        initial_state: начальное состояние регистров (список длины n).
        feedback_polynomial: полином обратной связи (список коэффициентов).
        p: простое число, определяющее поле GF(p).
        """
        self.n = n
        self.state = initial_state[:]
        self.feedback_polynomial = Polynomial(feedback_polynomial, p)
        self.p = p

    def shift(self):
        """Выполнить один такт сдвига."""
        # Вычислить новый бит на основе полинома обратной связи
        feedback_bit = sum(
            coeff * self.state[-i - 1] for i, coeff in enumerate(self.feedback_polynomial.coefficients)
        ) % self.p
        # Сдвинуть регистр
        self.state = [feedback_bit] + self.state[:-1]

    def multiply(self, polynomial):
        """Умножение текущего состояния на полином."""
        current_state_poly = Polynomial(self.state, self.p)
        result = current_state_poly * polynomial
        return result

    def divide(self, polynomial):
        """Деление текущего состояния на полином (возвращает остаток)."""
        current_state_poly = Polynomial(self.state, self.p)
        remainder = current_state_poly % polynomial
        return remainder

    def __str__(self):
        """Строковое представление состояния регистра."""
        return f"State: {self.state}"


# Параметры
n = 4  # Количество регистров
p = 2  # Поле GF(2)
initial_state = [1, 0, 1, 1]  # Начальное состояние
feedback_polynomial = [1, 0, 0, 1]  # Полином обратной связи x^3 + 1

# Создание регистра сдвига
sr = ShiftRegister(n, initial_state, feedback_polynomial, p)

print("Исходное состояние:")
print(sr)

# Умножение на полином
poly_to_multiply = Polynomial([1, 1], p)  # x + 1
result_multiply = sr.multiply(poly_to_multiply)
print("\nРезультат умножения на полином:")
print(result_multiply)

# Деление на полином
poly_to_divide = Polynomial([1, 0, 1], p)  # x^2 + 1
result_divide = sr.divide(poly_to_divide)
print("\nОстаток от деления на полином:")
print(result_divide)

# Выполнение одного такта сдвига
sr.shift()
print("\nСостояние после одного такта сдвига:")
print(sr)

Исходное состояние:
State: [1, 0, 1, 1]

Результат умножения на полином:
1x^4 + 1x^3 + 1x^2 + 1


KeyboardInterrupt: 

In [12]:
from collections import deque


DIRECT = True
REVERSE = False
# print(full_msg)


def label_content_bound(label: str, content: str, margin = 2):
    return max(len(label), len(content)) + margin


def label_and_content(label: str, content, filling: str = ' ') -> tuple[str, str]:
    printed_content = filling.join(str(bit) for bit in content)
    bound = label_content_bound(label, printed_content)
    label = label.center(bound)
    printed_content = printed_content.center(bound)
    return label, printed_content


def print_labels_and_content(full_msg: deque, state: deque, out_res: list) -> None:
    signal_label, printed_signal = label_and_content('Сигнал:', reversed(full_msg))
    state_label, printed_state = label_and_content('Регистры:', state, '|-|')
    out_label, printed_out = label_and_content('Out:', out_res[::-1])

    print(signal_label + state_label + out_label)
    print(printed_signal + printed_state + printed_out)
    print('')


def register_multiplication(n: int, q: int, begin_state: list, out_poly: list, msg: list):
    state = deque(begin_state)
    full_msg = deque(msg + [0] * n)
    full_len = len(full_msg)
    out_res = []
    leftovers = []
    
    print('Начальное состояние:')
    print_labels_and_content(full_msg, state, out_res)
    
    for i in range(1, full_len):
        new_signal = full_msg[0]
        full_msg.popleft()
        state.appendleft(new_signal)
        leftovers.append(state.pop())
        to_out = sum([look * pos for look, pos in zip(out_poly, state)]) % q
        out_res.append(to_out)
        print(f'Шаг {i}:')
        print_labels_and_content(full_msg, state, out_res)

In [ ]:
n = 3
q = 2
begin_state = [0, 0, 0]
out_poly = [1, 0, 1]
msg = [1, 1, 1]


register_multiplication(n, q, begin_state, out_poly, msg)

Начальное состояние:
   Сигнал:    Регистры:  Out: 
 0 0 0 1 1 1  0|-|0|-|0       

Шаг 1:
  Сигнал:   Регистры:  Out: 
 0 0 0 1 1  1|-|0|-|0   1   

Шаг 2:
 Сигнал:  Регистры:  Out: 
 0 0 0 1  1|-|1|-|0  1 1  

Шаг 3:
 Сигнал:  Регистры:   Out: 
  0 0 0   1|-|1|-|1  0 1 1 

Шаг 4:
 Сигнал:  Регистры:    Out:  
   0 0    0|-|1|-|1  1 0 1 1 

Шаг 5:
 Сигнал:  Регистры:     Out:   
    0     0|-|0|-|1  1 1 0 1 1 



In [55]:
def check_length(left, right, label: str):
    if len(left) != len(right):
        raise ValueError(f'Args of different length in {label}: {left} and {right}')


def GF_mul(left, right, q: int):
    check_length(left, right, 'GF_mul')
    return sum([l * r for l, r in zip(left, right)]) % q


def GF_sum(left: int, right: int, q: int):
    return (left + right) % q


def register_division(n: int, q: int, begin_state: list[int], out_poly: list[int], den: list[int], msg: list[int], max_len: int = 10):
    len_den = len(den)
    len_begin = len(begin_state)
    
    if len_den - 1 > len_begin:
        state = deque(begin_state + [0] * (len(den) - len(begin_state) - 1))
    elif len_begin + 1 > len_den:
        den = den.copy()
        den = den + [0] * (len_begin - len_den + 1)
        state = deque(begin_state)
    else:
        state = deque(begin_state)
    full_msg = deque(msg + [0] * max_len)
    len_state = len(state)
    len_out_poly = len(out_poly)
    if len_state > len_out_poly:
        out_poly = out_poly.copy()
        out_poly = out_poly + [0] * (len_state - len_out_poly)
    elif len_state < len_out_poly:
        state = state + [0] * (len_out_poly - len_state)
    full_len = len(full_msg)
    out_res = []
    leftovers = []
    target = [i for i, x in enumerate(den) if x][0]
    defer = [i for i, x in enumerate(out_poly) if x][0]
    den = den.copy()
    den[target] = 0
    print('Начальное состояние:')
    print_labels_and_content(full_msg, state, out_res)
    print(defer)
    for i in range(0, full_len):
        new_signal = full_msg[0]
        full_msg.popleft()
        state.appendleft(new_signal)
        state[target] = GF_sum(state[target], GF_mul(den, state, q), q)
        leftover = state.pop()
        leftovers.append(leftover)
        if i >= defer:
            to_out = GF_mul(out_poly, state, q)
            out_res.append(to_out)
        print(f'Шаг {i + 1}:')
        print_labels_and_content(full_msg, state, out_res)

In [57]:
n = 3
q = 2
begin_state = [0, 0, 0]
numerator = [0, 1, 1]
denominator = [0, 1]
msg = [1]

register_division(n, q, begin_state, numerator, denominator, msg, 5)

Начальное состояние:
   Сигнал:    Регистры:  Out: 
 0 0 0 0 0 1  0|-|0|-|0       

1
Шаг 1:
  Сигнал:   Регистры:  Out: 
 0 0 0 0 0  1|-|0|-|0       

Шаг 2:
 Сигнал:  Регистры:  Out: 
 0 0 0 0  0|-|1|-|0   1   

Шаг 3:
 Сигнал:  Регистры:  Out: 
  0 0 0   0|-|0|-|1  1 1  

Шаг 4:
 Сигнал:  Регистры:   Out: 
   0 0    0|-|0|-|0  0 1 1 

Шаг 5:
 Сигнал:  Регистры:    Out:  
    0     0|-|0|-|0  0 0 1 1 

Шаг 6:
 Сигнал:  Регистры:     Out:   
          0|-|0|-|0  0 0 0 1 1 

